In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import getpass
import os
import pandas as pd
import numpy as np

/var/folders/dl/8n__h2151hd3n3gtfckwb75w0000gn/T/ipykernel_93212/2684872567.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

In [4]:
#removed api key
llm = ChatGoogleGenerativeAI(model="gemini-pro", 
google_api_key="", safety_settings=safety_settings)

In [41]:
def evaluate_claim(claim):
    prompt = f"""please rate the veracity of the following claim on a scale from 0 to 5,
    with 0 being completely true and 5 being entirely false.
    Please ensure that the first character in your response is a single integer between    0 and 5,
    and explain your reasoning:
    {claim}"""

    response = llm.invoke(prompt).content

    rating = response[0]

    return response, int(rating)

## LIAR evaluation

In [6]:
df = pd.read_csv('liar_plus/test2.tsv', delimiter='\t', header = None)

df = df.drop(columns = [0])
df.rename({1: 'id', 2: 'label', 3: 'statement', 4: 'subject', 5: 'speaker', 6: 'job-title',
           7: 'state_info', 8: 'party_affiliation', 9: 'barely_true_counts', 10: 'false_counts',
           11: 'half_true_counts', 12: 'mostly_true_counts', 13: 'pants_on_fire_counts', 14: 'context',
           15: 'justification'
          }, axis = 1, inplace = True)

In [23]:
label_map = {'pants-fire': 5, 'false': 4, 'half-true': 3, 
             'barely-true': 2, 'mostly-true': 1, 'true': 0}

df['label'] = df['label'].replace(label_map)

/var/folders/dl/8n__h2151hd3n3gtfckwb75w0000gn/T/ipykernel_93212/398931798.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['label'].replace(label_map)


In [27]:
preds = []
labels = []

for index, row in df.iterrows():
    try:
        result = evaluate_claim(row['statement'])[1]

        preds.append(result)
        labels.append(row['label'])
    except:
        continue

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


In [42]:
results_df = pd.DataFrame({'predicted': preds, 'label': labels})

results_df['predicted'] = results_df['predicted'].astype(int)
(results_df['predicted'] == results_df['label']).mean()

0.17857142857142858

In [45]:
df['statement'].iloc[0]

'Building a wall on the U.S.-Mexico border will take literally years.'

In [44]:
#returns different labels every time (usually 0 or 2) -- maybe fix with fine tuning?
evaluate_claim(df['statement'].iloc[0])

('0\nBuilding a wall on the U.S.-Mexico border is a complex and time-consuming project that would take years to complete. According to experts, the construction of a wall along the entire U.S.-Mexico border would take at least 3.5 years and cost billions of dollars.',
 0)

In [40]:
results_df.dtypes

predicted    object
label         int64
dtype: object

In [51]:
(abs(results_df['predicted'].astype(int) - results_df['label'].astype(int)) <= 1).mean()

0.5124584717607974